In [1]:
import ipeadatapy as ipea
import pandas as pd
import numpy as np

In [2]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9603,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9604,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9605,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9606,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais (Atlas DH -...
9607,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas (Atlas DH ...


In [39]:
series.loc[series.NAME == "PIB - preços de mercado"]

,CODE,NAME
4428,SCN10_PIBN10,PIB - preços de mercado
4580,SCN104_PIBPMV104,PIB - preços de mercado


In [41]:
ipea.describe("SCN10_PIBN10")

,PIB - preços de mercado
Name,PIB - preços de mercado
Code,SCN10_PIBN10
Big Theme,Macroeconômico
Theme code,8
Source,Instituto Brasileiro de Geografia e Estatístic...
Source acronym,IBGE/SCN Anual
Comment,O produto interno bruto (PIB) é o total dos be...
Last update,2023-03-02T17:16:01-03:00
Frequency,Anual
Measure,R$


In [10]:
df = (
    pd.read_csv("deflator.csv")
    .rename({"DPCERG3A086NBEA": "deflator"}, axis = 1)
    .assign(DATE = lambda _: pd.to_datetime(_.DATE),
            deflator = lambda _: _.deflator / 100)
)

bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV"),
    "base_monetaria": ipea.describe("BM_M0FN"),
    "inflacao": ipea.describe("IGP_IGPDIG"),
    "cambio": ipea.describe("BM_ERCF"),
    "divida_externa": ipea.describe("BM_DEXR"),
    "pib_industria": ipea.describe("SCN10_VAINDN10"),
    "pib": ipea.describe("SCN10_PIBN10")
}


for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]
    bases[base]["dataframe"] = (
        ipea.timeseries(bases[base]["Code"]).reset_index()
        .rename({f"VALUE ({bases[base]['Measure']})": "VALUE"}, axis = 1)
        .assign(
            nome = base,
            unidade = bases[base]['Unit'], 
            medida = bases[base]['Measure'])
    )


df = (
    pd.concat([base["dataframe"] for base in bases.values()])
    .merge(df, how = "left")
    .pipe(lambda DF: DF.merge(DF
        .query("nome == 'cambio'")
        .rename({"VALUE": "cambio"}, axis = 1)
        .filter(["DATE", "cambio"]), 
    how = "left")
    )
)

bases_inalteradas = ["cambio", "base_monetaria"]

relatorio = (df
 .query("DATE.dt.year > 1950 and DATE.dt.year < 2000")
 .assign(
    unidade = lambda _: 
        np.where(_.unidade == 'milhões', 1000000,
        np.where(_.unidade.isna(), 1, np.nan)
    ),
    multiplicador = lambda _: np.where(np.isin(_.nome, bases_inalteradas), 1, 
        (_.unidade) / (
            (_.deflator ** ((_.medida == "US$") | (_.medida == "R$"))) * #Deflacionar os dolares
            (_.cambio ** (_.medida == "R$")) #Trocar reais por dolares
        )
    ),
    VALUE = lambda _: _.VALUE * _.multiplicador
).pivot_table(
    index = "YEAR",
    columns = "nome",
    values = "VALUE"
).reset_index()
 .rename_axis("", axis = 1)
)

In [11]:
relatorio.to_csv("data.csv")
relatorio.head()

,YEAR,base_monetaria,cambio,divida_externa,exportacao,importacao,inflacao,pib,pib_industria
0,1951,1.589091e-11,6.683636e-15,4.033910e+09,1.249736e+10,1.218505e+10,12.337655,1.340668e+11,3.124893e+10
1,1952,1.869091e-11,6.683636e-15,4.416753e+09,9.817355e+09,1.190897e+10,12.716743,1.545015e+11,3.480238e+10
2,1953,2.214545e-11,1.981818e-14,7.919371e+09,1.051808e+10,7.820998e+09,20.512819,6.137104e+10,1.455603e+10
3,1954,2.690909e-11,2.618182e-14,8.104079e+09,1.058298e+10,9.585899e+09,25.864923,6.316725e+10,1.502951e+10
4,1955,3.192727e-11,2.345455e-14,9.415497e+09,9.606142e+09,7.449987e+09,12.145735,8.525248e+10,2.045767e+10


In [9]:
bases["base_monetaria"]

{'Name': 'Base monetária restrita -M0 - fim de período',
 'Code': 'BM_M0FN',
 'Big Theme': 'Macroeconômico',
 'Theme code': 3,
 'Source': 'Banco Central do Brasil, Notas Econômico-financeiras para a Imprensa, Política Monetária e Operações de Crédito do SFN (Bacen/Notas Imprensa/Moeda)',
 'Source acronym': 'Bacen/Notas Imprensa/Moeda',
 'Comment': 'A base monetária restrita é a soma do papel-moeda emitido (PME) com as reservas bancárias. É conhecida como M0.As reservas bancárias representam a “conta-corrente” das instituições financeiras no Banco Central, sendo um instrumento de liquidação necessário para que sejam efetuadas as transações monetárias entre a Autoridade Monetária e os demais agentes econômicos (sistema financeiro, governo e setor externo). Mais informações: <a href="../doc/NT 48_Dstat_Dimob_novembro_2018.pdf"> Nota Técnica do Banco Central do Brasil nª48</a>.',
 'Last update': '2023-01-27T18:50:00-03:00',
 'Frequency': 'Anual',
 'Measure': 'R$',
 'Unit': 'milhões',
 'Sta